In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import ndimage
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from glob import glob


import tensorflow as tf
import keras
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D,BatchNormalization
from keras.layers import Conv2DTranspose, Dropout,GlobalAveragePooling2D

from keras.layers import concatenate, GaussianNoise,UpSampling2D
from keras.models import Model
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from keras import regularizers
from keras.optimizers import Adam
from keras import backend as K
from keras.losses import binary_crossentropy,categorical_crossentropy



import cv2
import os



In [ ]:
train_base = pd.read_csv('../input/severstal-steel-defect-detection/train.csv')
print(train_base.shape)
train_base.head(3)
#12568


In [ ]:
# image id and class id are two seperate entities and it makes it easier to split them up in two columns
#train_base = train_base[train_base['EncodedPixels'].notnull()].reset_index(drop=True)
train_base['ImageId'] = train_base['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
train_base['ClassId'] = train_base['ImageId_ClassId'].apply(lambda x: x.split('_')[1])
train_base['hasMask'] = ~ train_base['EncodedPixels'].isna()
train_base.head(2)


In [ ]:
mask_count_df = train_base.groupby('ImageId').agg(np.sum).reset_index()
mask_count_df.head()

In [ ]:
base = pd.merge(train_base,mask_count_df, how='left',  left_on='ImageId', right_on='ImageId',)
base.head()

In [ ]:
base_full = base[base['hasMask_y']>0]
base_full.fillna(-1, inplace = True)
base_full = pd.DataFrame(base_full).reset_index()
base_full.head() #samples with at least one mask

In [ ]:
train_dir = '../input/severstal-steel-defect-detection/' # directory of training images
pretrained_model_path = '../input/severstal-pretrained-model/ResUNetSteel_z.h5' # path of pretrained model
model_save_path = './ResUNetSteel_w800e50_z.h5' # path of model to save
train_image_dir = os.path.join(train_dir, 'train_images') # 

In [ ]:
# create a dict of all the masks
masks = {}
for index, row in base_full[base_full['EncodedPixels']!=-1].iterrows():
    masks[row['ImageId_ClassId']] = row['EncodedPixels']

train_image_ids = base_full['ImageId'].unique()

In [ ]:
base_full[base_full['EncodedPixels'] == -1]['ClassId'].value_counts()

In [ ]:
5865/(5865+801)


In [ ]:
base_full[base_full['EncodedPixels'] != -1]['ClassId'].value_counts()

In [ ]:
base_def_1 = base_full[(base_full['EncodedPixels'] !=-1) & (base_full['hasMask_y'] == 1)]
base_def_1.reset_index(drop = True, inplace = True)
base_def_1.head(2)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_test = StratifiedShuffleSplit(n_splits = 1, test_size = 0.3, random_state = 222)
X = base_def_1['ImageId']
y = base_def_1['ClassId']
for train_index, test_index in train_test.split(X,y):
    X_train, X_val = X[train_index], X[test_index]
    Y_train, Y_val = y[train_index], y[test_index]
X_train_1 = X_train.reset_index(drop = True)   
X_val_1 = X_val.reset_index(drop = True)
#Y_val = Y_val.reset_index(drop = True)



In [ ]:
multidef = base_full[(base_full['EncodedPixels'] !=-1) & (base_full['hasMask_y'] > 1)]
image_ids = multidef['ImageId'].unique()
X_train, X_val = train_test_split(image_ids, test_size=0.3, random_state=222)
#X_val, X_test = train_test_split(X_val, test_size=0.4, random_state=222)

In [ ]:
X_train = np.concatenate([X_train_1, X_train])
X_val = np.concatenate([X_val_1, X_val])
#X_test = np.concatenate([X_test_1, X_test])
print(len(X_train), len(X_val))

In [ ]:
img_h = 128
img_w = 800

In [ ]:
# from https://www.kaggle.com/robertkag/rle-to-mask-converter
def rle_to_mask(rle_string,height,width):
    '''
    convert RLE(run length encoding) string to numpy array

    Parameters: 
    rleString (str): Description of arg1 
    height (int): height of the mask
    width (int): width of the mask 

    Returns: 
    numpy.array: numpy array of the mask
    '''
    rows, cols = height, width
    if rle_string == -1:
        return np.zeros((height, width))
    else:
        rleNumbers = [int(numstring) for numstring in rle_string.split(' ')]
        rlePairs = np.array(rleNumbers).reshape(-1,2)
        img = np.zeros(rows*cols,dtype=np.uint8)
        for index,length in rlePairs:
            index -= 1
            img[index:index+length] = 255
        img = img.reshape(cols,rows)
        img = img.T
        return img
# Thanks to the authors of: https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
def mask_to_rle(mask):
    '''
    Convert a mask into RLE
    
    Parameters: 
    mask (numpy.array): binary mask of numpy array where 1 - mask, 0 - background

    Returns: 
    sring: run length encoding 
    '''
    pixels= mask.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
##Data Generator
#To push the data to our model, we will create a custom data generator. A generator lets us load data progressively, instead of loading it all into memory at once. A custom generator allows us to also fit in more customization during the time of loading the data. As the model is being procssed in the GPU, we can use a custom generator to pre-process images via a generator. At this time, we can also take advantage multiple processors to parallelize our pre-processing.

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, list_ids, labels, image_dir, batch_size=10,
                 img_h=img_h, img_w=img_w, shuffle=True):
        
        self.list_ids = list_ids
        self.labels = labels
        self.image_dir = image_dir
        self.batch_size = batch_size
        self.img_h = img_h
        self.img_w = img_w
        self.shuffle = shuffle
        self.on_epoch_end()
    
    def __len__(self):
        'denotes the number of batches per epoch'
        return int(np.floor(len(self.list_ids)) / self.batch_size)
    
    def __getitem__(self, index):
        'generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # get list of IDs
        list_ids_temp = [self.list_ids[k] for k in indexes]
        # generate data
        X, y = self.__data_generation(list_ids_temp)
        # return data 
        return X, y
    
    def on_epoch_end(self):
        'update ended after each epoch'
        self.indexes = np.arange(len(self.list_ids))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, list_ids_temp):
        'generate data containing batch_size samples'
        X = np.empty((self.batch_size, self.img_h, self.img_w, 1))
        y = np.empty((self.batch_size, self.img_h, self.img_w, 5))
        
        for idx, id in enumerate(list_ids_temp):
            file_path =  os.path.join(self.image_dir, id)
            image = cv2.imread(file_path, 0)
            image_resized = cv2.resize(image, (self.img_w, self.img_h))
            image_resized = np.array(image_resized, dtype=np.float64)
            # standardization of the image
            image_resized -= image_resized.mean()
            image_resized /= image_resized.std()
            
            mask = np.empty((self.img_h, self.img_w, 5))
            mark =  np.empty((self.img_h, self.img_w))
            
            for idm, image_class in enumerate(['1','2','3','4']):
                rle = self.labels.get(id + '_' + image_class)
                # if there is no mask create empty mask
                if rle is None:
                    class_mask = np.zeros((1600, 256))
                else:
                    class_mask = rle_to_mask(rle, width=1600, height=256)
             
                class_mask_resized = cv2.resize(class_mask, (self.img_w, self.img_h))
                mark += class_mask_resized
                mask[...,idm] = class_mask_resized
            mark = (mark==0).astype(int)    
            mask[...,4] = mark
            X[idx,] = np.expand_dims(image_resized, axis=2)
            y[idx,] = mask
        
        # normalize Y
        y = (y > 0).astype(int)
            
        return X, y

In [ ]:
batch_size = 10
params = {'img_h': img_h,
          'img_w': img_w,
          'image_dir': train_image_dir,
          'batch_size': batch_size,
          'shuffle': True}

# Get Generators
training_generator = DataGenerator(X_train, masks, **params)
validation_generator = DataGenerator(X_val, masks, **params)

In [ ]:
#Callbacks
early_stopping_monitor = EarlyStopping(monitor='dice_coef',patience = 5)
check = ModelCheckpoint(
    'model.h5', 
    monitor='val_loss', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto'
)

red_rl = ReduceLROnPlateau(factor=0.3, patience=3, min_lr=0.000001)

In [ ]:
def dice_single_channel( y_true,y_pred, eps = 1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + eps) / (K.sum(y_true_f) + K.sum(y_pred_f) + eps)



In [ ]:
def dice_coef(y_true,y_pred):
    treshhold = 0.5
    batch_size = 10
    channel_num = 5
    refer_pos = [2,2,1,1.5]
    
    dice_batch = []
    for i in range(batch_size):
        dice = []
        for j in range(channel_num):
            pred =  y_pred[i,:,:,j]
            tr = y_true[i,:,:,j]
            if tr is np.zeros:
                dice.append(dice_single_channel(tr, pred))
            else:
                if j==0:
                     dice.append(2*dice_single_channel(tr, pred))
                if j==1:
                     dice.append(2.5*dice_single_channel(tr, pred))        
                if j == 3:
                     dice.append(1.5*dice_single_channel(tr, pred))
                else:
                      dice.append(dice_single_channel(tr, pred))
        dice =sum(dice)/channel_num
        dice_batch.append(dice)
    return  sum(dice_batch)/batch_size

In [ ]:
def cce(y_true, y_pred):
    return 0.7*categorical_crossentropy(y_true, y_pred) - 0.25* dice_coef(y_true, y_pred)

In [ ]:
def model_Unet():


    inputs = Input((None, None, 1))
    bnorm1 = BatchNormalization()(inputs)
    conv1 = Conv2D(32, (3, 3),init='he_uniform', W_regularizer=regularizers.l2(0.0001), activation='relu', padding='same')(bnorm1)
    conv1 = Conv2D(32, (3, 3), activation='relu', W_regularizer=regularizers.l2(0.0001), padding='same')(conv1)
    #drop1 = Dropout(0.25)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3),activation='relu', padding='same')(pool1)
    bnorm2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3),activation='relu', padding='same')(bnorm2)

    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    bnorm3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(bnorm3)

    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    bnorm4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256, (3, 3),  activation='relu', padding='same')(bnorm4)

    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    bnorm5 = BatchNormalization()(conv5)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(bnorm5)


    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    bnorm6 = BatchNormalization()(conv6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(bnorm6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    bnorm7 = BatchNormalization()(conv7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(bnorm7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    bnorm8 = BatchNormalization()(conv8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(bnorm8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    bnorm9 = BatchNormalization()(conv9)
    conv9 = Conv2D(32, (3, 3),  activation='relu', padding='same')(bnorm9)

    conv10 = Conv2D(5, (1, 1), activation='softmax')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])
    return model



In [ ]:
model = model_Unet()
model.compile(Adam(0.0001),
              loss=cce, metrics=[dice_coef])

In [ ]:

history = model.fit_generator(generator=training_generator, validation_data=validation_generator, epochs=15, verbose=1,
                             shuffle=True,callbacks = [check,red_rl])


In [ ]:
model.save('modelUnet-cat+dice.h5')

Block for prediction

In [ ]:
# return tensor in the right shape for prediction 
def decode_test_image(img_dir, img_h, img_w, channels=1):

    X = np.empty((1, img_h, img_w, channels))
    # Store sample
    image = cv2.imread(img_dir, 0)
    image_resized = cv2.resize(image, (img_w, img_h))
    image_resized = np.array(image_resized, dtype=np.float64)
    # normalize image
    image_resized -= image_resized.mean()
    image_resized /= image_resized.std()
    
    X[0,] = np.expand_dims(image_resized, axis=2)

    return X

In [ ]:
samp = pd.read_csv('../input/severstal-steel-defect-detection/sample_submission.csv')
samp['ImageId'] = samp['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
X_test = samp['ImageId'].unique()

In [ ]:
test_dir = '../input/severstal-steel-defect-detection/test_images/'
test_files = []
for idx, id in enumerate(X_test):
    file_path = os.path.join(test_dir, id)
    test_files.append(file_path)

In [ ]:
# this is an awesome little function to remove small spots in our predictions

from skimage import morphology

def remove_small_regions(img, size):
    """Morphologically removes small (less than size) connected regions of 0s or 1s."""
    img = morphology.remove_small_objects(img, size)
    img = morphology.remove_small_holes(img, size)
    return img

In [ ]:
submission = []

# a function to apply all the processing steps necessery to each of the individual masks
def process_pred_mask(pred_mask):
    
    pred_mask = cv2.resize(pred_mask.astype('float32'),(1600, 256))
    pred_mask = (pred_mask > .5).astype(int)
    pred_mask = remove_small_regions(pred_mask, 0.02 * np.prod(512)) * 255
    pred_mask = mask_to_rle(pred_mask)
    
    return pred_mask

# loop over all the test images
for f in test_files:
    # get test tensor, output is in shape: (1, 256, 512, 3)
    test = decode_test_image(f, img_h, img_w,1) 
    # get prediction, output is in shape: (1, 256, 512, 4)
    pred_masks = model.predict(test) 
    # get a list of masks with shape: 256, 512
    pred_masks = [pred_masks[0][...,i] for i in range(0,4)]
    # apply all the processing steps to each of the mask
    pred_masks = [process_pred_mask(pred_mask) for pred_mask in pred_masks]
    # get our image id
    idx = f.split('/')[-1]
    # create ImageId_ClassId and get the EncodedPixels for the class ID, and append to our submissions list
    [submission.append((idx+'_%s' % (k+1), pred_mask)) for k, pred_mask in enumerate(pred_masks)]


In [ ]:
submission_test = pd.DataFrame(submission, columns=['ImageId_ClassId', 'EncodedPixels'])
submission_test[ submission_test['EncodedPixels'] != ''].head()

submission_test.to_csv('./submission.csv', index=False)

In [ ]:
submission_test.head()

In [ ]:
from IPython.display import FileLink, FileLinks
FileLinks('.')

In [ ]:
test_dir = '../input/severstal-steel-defect-detection/train_images/'
test_files = []
for idx, id in enumerate(X_val):
    file_path = os.path.join(test_dir, id)
    test_files.append(file_path)

In [ ]:
for f in test_files:
    # get test tensor, output is in shape: (1, 256, 512, 3)
    test = decode_test_image(f, img_h, img_w,1) 
    # get prediction, output is in shape: (1, 256, 512, 4)
    pred_masks = model.predict(test) 
    # get a list of masks with shape: 256, 512
    pred_masks = [pred_masks[0][...,i] for i in range(0,4)]
    # apply all the processing steps to each of the mask
    pred_masks = [process_pred_mask(pred_mask) for pred_mask in pred_masks]
    # get our image id
    idx = f.split('/')[-1]
    # create ImageId_ClassId and get the EncodedPixels for the class ID, and append to our submissions list
    [submission.append((idx+'_%s' % (k+1), pred_mask)) for k, pred_mask in enumerate(pred_masks)]


In [ ]:
submission_val = pd.DataFrame(submission, columns=['ImageId_ClassId', 'EncodedPixels'])
submission_val[ submission_val['EncodedPixels'] != ''].head()

submission_val.to_csv('./submission_val.csv', index=False)

In [ ]:
submission_unet = submission_val

In [ ]:
submission_unet.fillna(-1, inplace = True)
submission_unet.head()

In [ ]:
def dice_coef_q(y_true, y_pred):
    
    intersection = sum(y_true * y_pred)
    return (2. * intersection + 1) / (sum(y_true) + sum(y_pred) + 1)



def dice_float(base,num):
   
    y_true = rle2mask(base['rle_true'][num])
    y_pred = rle2mask(base['rle_pred'][num])
    
    dice = dice_coef_q(y_true, y_pred)
    return dice

    """
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        return dice.eval()
       
    """ 
def dice_column(base):
    dice_column = []
    for i in range(base.shape[0]):
        dice = dice_float(base, i)
        dice_column.append(dice)
    return dice_column
    

In [ ]:
unet_result = submission_unet.merge(base_full, left_on = 'ImageId_ClassId', right_on = 'ImageId_ClassId' )
unet_result.rename(columns={'EncodedPixels_x': 'rle_pred', 'EncodedPixels_y': 'rle_true'}, inplace=True)
unet_result = unet_result[['ImageId_ClassId','rle_pred','rle_true','ImageId','ClassId']]
unet_result.head()

In [ ]:
def rle2mask(rle):
    # CONVERT RLE TO MASK 
    if rle== -1: 
        return np.zeros((256*1600),dtype=np.uint8)
    else:
        height= 256
        width = 1600
        mask= np.zeros( width*height ,dtype=np.uint8)

        array = np.asarray([int(x) for x in rle.split()])
        starts = array[0::2]-1
        lengths = array[1::2]    
        for index, start in enumerate(starts):
            mask[int(start):int(start+lengths[index])] = 1

        return mask.reshape( (height*width), order='F' )


In [ ]:
unet_total = dice_column(unet_result)
unet_result['dice'] = unet_total
unet_result.head(5)

In [ ]:
unet_result.to_csv('unet_result_cat.csv')

In [ ]:
unet_result['dice'].mean()

In [ ]:
unet_result[['ClassId','dice']].groupby('ClassId').agg({'mean','count'})

In [ ]:
unet_nan = unet_result[unet_result['rle_true']==-1]
unet_nan[['ClassId','dice']].groupby('ClassId').agg('mean')

In [ ]:
unet_nan = unet_result[unet_result['rle_pred']==-1]
unet_nan[['ClassId','dice']].groupby('ClassId').agg('mean')

In [ ]:
unet_nan = unet_result[unet_result['rle_true']!=-1]
unet_nan[['ClassId','dice']].groupby('ClassId').agg('mean')

In [ ]:
unet_result.to_csv('unet_reslt.csv')

In [ ]:
Files(,)